In [1]:
import os
import time
import ctypes
import sys

# from ctypes import *

# pylablib
ref: [thorlabs kinesis](https://pylablib.readthedocs.io/en/latest/devices/Thorlabs_kinesis.html#thorlabs-apt-kinesis-devices)

In [2]:
from typing import Any
import functools
from pylablib.devices import Thorlabs
import time
import numpy as np


class FP_system_control(object):
    """
    Fourier Ptychography system control Class
    """

    def __init__(
        self,
        scale=(34555.0, 772970.0, 264.0),
        move_min_velocity=0,
        move_max_velocity=1,
        acceleration=1,
        home_direction="forward",
        *args,
        **kwargs,
    ) -> None:
        """
        initialize the whole system: translation stages and cameras
        """

        self.motor_SN = [
            device[0] for device in Thorlabs.list_kinesis_devices()
        ]  # translation stage serial number
        print(f"detected translation stage(s) S/N:\n{self.motor_SN}\n")

        self.stage = {}  # translation stage objects
        self.axis = [chr(97 + 23 + i) for i in range(len(self.motor_SN))] + [
            str("\u03B8")
        ]  # axis name 'x', 'y', 'z', and 'θ' for renaming translation stages

        ## config translation stage parameters
        self.scale = scale  # device unit in [1 mm / scale]
        self.move_min_velocity = move_min_velocity
        self.move_max_velocity = move_max_velocity
        self.acceleration = acceleration
        self.home_direction = home_direction

        print("start to initialize all translation stages:")
        self.init_all_stage()  # initialize all translation stages

    def __str__(self) -> str:
        """
        brief introduction
        """
        return "Fourier Ptychography camera system control"

    def __call__(self, *args: Any, **kwds: Any) -> Any:
        """
        decorator
        """
        pass

    @staticmethod
    def check_value(dict, value):
        if value in dict.values():
            return True
        else:
            return False

    def _search_name(func):

        @functools.wraps(func)
        def wrapper(self, *name_list, **new_name_dict):

            name_list = list(name_list)
            if name_list == []:  # choose all stages if name_list == []
                name_list = list(self.stage.keys())

            exist_list = list(
                map(
                    self.check_value,
                    [self.stage.copy() for _ in range(len(name_list))],
                    name_list,
                )
            )

            if new_name_dict == {}:
                return func(
                    self, name_list, exist_list
                )  # change_stage_name(self, old_name, new_name="x")
            else:
                return func(self, name_list, list(new_name_dict.values()), exist_list)

        return wrapper

    def init_all_stage(
        self,
    ) -> None:
        """
        initialize all detected translation stages
        """

        for SN in self.motor_SN:
            print(f"initializing {SN}:", end=" ")
            self.stage[SN] = Thorlabs.KinesisMotor(
                SN, self.scale
            )  # connect to stage and set scale. [1 mm / scale]

            ## config moving velocity
            self.stage[SN].setup_velocity(
                min_velocity=self.move_min_velocity,
                max_velocity=self.move_max_velocity,
                acceleration=self.acceleration,
            )

            ## config home mode
            self.stage[SN].setup_homing(
                velocity=self.move_max_velocity,
                home_direction=self.home_direction,
                limit_switch=self.home_direction,
            )

            ## config jog mode
            self.stage[SN].setup_jog(
                min_velocity=self.move_min_velocity,
                max_velocity=self.move_max_velocity,
                acceleration=self.acceleration,
            )

            self.stage[SN].home(force=True)  # home the stage
            self.stage[SN].wait_for_home()  # wait for "home"
            time.sleep(2)  # wait to be stable
            print("initialization finished.", end=" ")
            print(f"current position: {self.stage[SN].get_position()} [mm]")

        # time.sleep(2)  # wait for the last translation stage to finish "home"
        print("translation stage(s) initialization finished! \n")

    @_search_name
    def change_stage_name(
        self, old_name_list: list = [], new_name_list: list = [], exit_list: list = []
    ) -> None:
        """
        change translation stage's name
        """
        operate_name_list = []
        for old_name, new_name, exit_flag in zip(
            old_name_list, new_name_list, exit_list
        ):
            if exit_flag:
                self.stage[new_name] = self.stage.pop(old_name)
                operate_name_list.append(old_name)
            else:
                print(f"cannot find stage {old_name}")

        print(f"{operate_name_list} finished renaming!\n")
        self.get_all_stage_name()

    @_search_name
    def close_stages(self, name_list: list = [], exit_list: list = []) -> None:
        """
        turn off the connection to translation stage(s)
        """
        operate_name_list = []
        for name, exit_flag in zip(name_list, exit_list):
            if exit_flag:
                self.stage[name].close()
                self.stage.pop(name)
                operate_name_list.append(name)
                # print(f"closed translation stage {name}")
            else:
                print(f"cannot find stage {name}")

        self.motor_SN = [device[0] for device in Thorlabs.list_kinesis_devices()]
        print(f"{operate_name_list} finished close!\n")
        self.get_all_stage_name()

    @_search_name
    def get_stage_position(self, name_list: list = [], exit_list: list = []) -> None:
        """
        acquire translation stage's position in [mm]
        """
        for name, exit_flag in zip(name_list, exit_list):
            if exit_flag:
                print(
                    f"stage {name}'s current position: {self.stage[name].get_position()} [mm]"
                )
            else:
                print(f"cannot find stage {name}")

            print()

    @_search_name
    def get_stage_full_info(self, name_list: list = [], exit_list: list = []) -> None:
        """
        acquire the full info of translation stage
        """
        for name, exit_flag in zip(name_list, exit_list):
            if exit_flag:
                print(
                    f"stage {name}'s current position: {self.stage[name].get_position()} [mm]"
                )
            else:
                print(f"cannot find stage {name}")

            print()

    @_search_name
    def home_stage(self, name_list: list = [], exit_list: list = []) -> None:
        """
        home translation stage(s)
        """
        operate_name_list = []
        for name, exit_flag in zip(name_list, exit_list):
            if exit_flag:
                self.stage[name].home(force=True)  # home the stage
                self.stage[name].wait_for_home()  # wait for "home"
                time.sleep(2)  # wait to be stable
            else:
                print(f"cannot find stage {name}")

            print(f"{operate_name_list} finished homing!\n")
            self.get_stage_position()

    @_search_name
    def move_stage(
        self, name_list: list = [], pos_list: list = [], exit_list: list = []
    ) -> None:
        """
        home translation stage(s)
        """
        operate_name_list = []
        for name, pos, exit_flag in zip(name_list, pos_list, exit_list):
            if exit_flag:
                self.stage[name].move_to(pos)
                self.stage[name].wait_move()  # wait for moving
                operate_name_list.append(name)
            else:
                print(f"cannot find stage {name}")
        time.sleep(2)  # wait to be stable
        print(f"{operate_name_list} finished homing!\n")
        self.get_stage_position()

    def get_all_stage_name(self) -> None:
        """
        get all connected stage(s) name
        """
        print("currently connected translation stage(s):")
        for name, sn_num in zip(
            list(self.stage.keys()),
            [device[0] for device in Thorlabs.list_kinesis_devices()],
        ):
            print(f"S/N {sn_num}: {name}")

        print()

    def move_cam_xy(self, pos: "(x,y) coordiante" = []) -> None:
        """
        move the camera(s) to a (x,y) position

        define the coordinate system based on the back view of camera(s) as follows
        y
        ^
        |
        |
        o------->x
        """

        pass

    def move_cam_xyz(self, pos: "(x,y,z) coordiante" = []) -> None:
        """
        move the camera(s) to a (x,y,z) position

        define the coordinate system based on the back view of camera(s) as follows
        y(+)  z(+)
          ^  /
          | /
          |/
          o------->x(+)
        """

        pass

    def cam_capture(self, frame_num: list = [], exp_time: list = []) -> None:
        pass

In [3]:
FP_xy_1cam = FP_system_control()

detected translation stage(s) S/N:
['27266045', '27257923']

start to initialize all translation stages:
initializing 27266045: finished. current position: 2.893937201562726e-05 [mm]
initializing 27257923: finished. current position: 0.0 [mm]
translation stage(s) initialization finished! 



In [113]:
import functools


def outer(test):
    def decorator(func):
        @functools.wraps(func)
        def wrapper(value):
            # 原函数运行前
            # Do something
            value = func(test, value)
            # 原函数运行后
            # Do something
            return value

        return wrapper

    return decorator


@outer(2)
def add(x, y):
    return x + y

In [5]:
def check_value(dict, value):
    if value in dict.values():
        return True
    else:
        return False

In [22]:
old_name_list + [6]

[1, 2, 3, 6]

In [18]:
new_name_dict = {1: "one", 2: "two", 3: 3}
old_name_list = list(range(1, 4))
exit_list = list(
    map(
        check_value,
        [new_name_dict.copy() for _ in range(len(old_name_list))],
        old_name_list,
    )
)

In [10]:
[name for flag, name in zip(exit_list, old_name_list) if flag]

[3]

In [83]:
for old_name, new_name in zip(old_name_list, list(new_name_dict.values())):
    print(old_name, new_name)

1 one
2 two
3 three


In [70]:
a = [1, 2, 3]
b = [4, 5, 6]

In [74]:
list(zip(*zip(a, b)))[0]

(1, 2, 3)

In [41]:
FP_xy_1cam.get_stage_position()

stage 27266045's current position: 2.893937201562726e-05 [mm]
stage 27257923's current position: 0.0 [mm]


In [43]:
FP_xy_1cam.change_stage_name(old_name="27266045", new_name="x")
FP_xy_1cam.change_stage_name(old_name="27257923", new_name="y")

KeyError: '27266045'

In [30]:
FP_xy_1cam.close_stages()

start close the following translation stages: ['x', 'y']
closed translation stage x
closed translation stage y
closed all translation stages!



In [17]:
str("\u03B8")

'θ'

In [190]:
[device[0] for device in Thorlabs.list_kinesis_devices()]

['27266045', '27257923']

In [163]:
x_stage = Thorlabs.KinesisMotor(
    Thorlabs.list_kinesis_devices()[0][0], scale=(34555.0, 772970.0, 264.0)
)  # [1 mm / scale]


y_stage = Thorlabs.KinesisMotor(
    Thorlabs.list_kinesis_devices()[1][0], scale=(34555.0, 772970.0, 264.0)
)  # [1 mm / scale]


# modfied in class KinesisMotor(KinesisDevice):


# if stage in {"MTS25-Z8","MTS50-Z8"}:


#             return 34555E0,"mm"

In [164]:
x_stage.setup_velocity(min_velocity=0, max_velocity=1, acceleration=1)
x_stage.setup_homing(velocity=1, home_direction="forward", limit_switch="forward")
x_stage.setup_jog(min_velocity=0, max_velocity=1, acceleration=1)

y_stage.setup_velocity(min_velocity=0, max_velocity=1, acceleration=1)
y_stage.setup_homing(velocity=1, home_direction="forward", limit_switch="forward")
y_stage.setup_jog(min_velocity=0, max_velocity=1, acceleration=1)

TJogParams(mode='step', step_size=0.4999855303139922, min_velocity=0.0, acceleration=1.0, max_velocity=1.0, stop_mode='profiled')

In [165]:
print(x_stage.get_full_info())
print(y_stage.get_full_info())

{'velocity_parameters': TVelocityParams(min_velocity=0.0, acceleration=1.0, max_velocity=1.0), 'jog_parameters': TJogParams(mode='step', step_size=0.4999855303139922, min_velocity=0.0, acceleration=1.0, max_velocity=1.0, stop_mode='profiled'), 'homing_parameters': THomeParams(home_direction='forward', limit_switch='forward', velocity=1.0, offset_distance=1.0), 'gen_move_parameters': TGenMoveParams(backlash_distance=0.05000723484300391), 'limit_switch_parameters': TLimitSwitchParams(hw_kind_cw='make', hw_kind_ccw='make', hw_swapped=False, sw_position_cw=3.0, sw_position_ccw=1.0, sw_kind='ignore'), 'kcube_trigio_parameters': TKCubeTrigIOParams(trig1_mode='off', trig1_pol=True, trig2_mode='off', trig2_pol=True), 'kcube_trigpos_parameters': TKCubeTrigPosParams(start_fw=0.0, step_fw=0.0, num_fw=1, start_bk=0.0, step_bk=0.0, num_bk=1, width=0.05, ncycles=1), 'position': 0.0, 'status': ['sw_bk_lim', 'connected', 'homed', 'digio1', 'digio2', 'power_ok', 'enabled'], 'cls': 'KinesisMotor', 'conn

In [184]:
x_stage.home(force=True)  # home the stage
y_stage.home(force=True)  # home the stage

x_stage.wait_for_home()  # wait until homing is done
y_stage.wait_for_home()  # wait until homing is done

In [183]:
print(x_stage.get_position())
print(y_stage.get_position())

-25.000086818116046
-25.0


In [185]:
x_stage.move_to(-25)
x_stage.wait_move()
y_stage.move_to(-25)
y_stage.wait_move()

In [204]:
x_stage.close()
y_stage.close()

# thorlabs-kinesis

In [2]:
if sys.version_info < (3, 8):
    os.chdir(r"C:\Program Files\Thorlabs\Kinesis")
else:
    os.add_dll_directory(r"C:\Program Files\Thorlabs\Kinesis")

lib: CDLL = cdll.LoadLibrary("Thorlabs.MotionControl.KCube.DCServo.dll")

In [3]:
# Uncomment this line if you are using simulations
lib.TLI_InitializeSimulations()

# Set constants
serial_num = c_char_p(b"27000001")

if lib.TLI_BuildDeviceList() == 0:
    lib.CC_Open(serial_num)
    lib.CC_StartPolling(serial_num, c_int(200))

    if lib.CC_CanHome(serial_num):
        lib.CC_Home(serial_num)

        lib.CC_RequestStatusBits(serial_num)
        print(lib.CC_GetStatusBits(serial_num))

        while lib.CC_GetPositionCounter(serial_num) != 0:
            pass

        time.sleep(0.1)

        print("home operation completed!")

        lib.CC_RequestStatusBits(serial_num)
        print(lib.CC_GetStatusBits(serial_num))

    # # Set up the device to convert real units to device units
    # STEPS_PER_REV = c_double(34555)  # for the MTS50-Z8
    # gbox_ratio = c_double(1.0)  # gearbox ratio
    # pitch = c_double(1.0)

    # # Apply these values to the device
    # lib.CC_SetMotorParamsExt(serial_num, STEPS_PER_REV, gbox_ratio, pitch)

    # # Get the device's current position in dev units
    # lib.CC_RequestPosition(serial_num)
    # time.sleep(0.2)
    # dev_pos = c_int(lib.CC_GetPosition(serial_num))

    lib.CC_Close(serial_num)


# Uncomment this line if you are using simulations
lib.TLI_UninitializeSimulations()

0
home operation completed!
-2147482624


2

# thorlabs_apt_device

In [2]:
from thorlabs_apt_device import TDC001, KDC101

In [30]:
stage = TDC001("COM6", home=False)
time.sleep(1)
while (
    stage.status["moving_forward"] == True
    or stage.status["moving_reverse"] == True
    or stage.status["homing"] == True
):
    pass

In [34]:
stage.home()
time.sleep(1)
while (
    stage.status["moving_forward"] == True
    or stage.status["moving_reverse"] == True
    or stage.status["homing"] == True
):
    pass

In [227]:
stage.stop(True)

In [35]:
stage.status

{'position': 0,
 'enc_count': 0,
 'velocity': 0,
 'forward_limit_switch': False,
 'reverse_limit_switch': False,
 'moving_forward': False,
 'moving_reverse': False,
 'jogging_forward': False,
 'jogging_reverse': False,
 'motor_connected': True,
 'homing': False,
 'homed': True,
 'tracking': False,
 'interlock': False,
 'settled': False,
 'motion_error': False,
 'motor_current_limit_reached': False,
 'channel_enabled': True,
 'msg': 'mot_get_dcstatusupdate',
 'msgid': 1169,
 'source': 80,
 'dest': 1,
 'chan_ident': 1,
 'forward_limit_soft': True,
 'reverse_limit_soft': False,
 'initializing': False,
 'instrument_error': False,
 'overtemp': False,
 'voltage_fault': False,
 'commutation_error': False,
 'digital_in_1': False,
 'digital_in_2': False,
 'digital_in_3': False,
 'digital_in_4': False,
 'encoder_fault': False,
 'overcurrent': False,
 'current_fault': False,
 'power_ok': True,
 'active': False,
 'error': False}

In [32]:
# Flash the LED on the device to identify it
stage.identify()
stage.set_velocity_params(264, 772970)

stage.move_absolute(34555 * 10)
time.sleep(1)
while (
    stage.status["moving_forward"] == True
    or stage.status["moving_reverse"] == True
    or stage.status["homing"] == True
):
    pass
time.sleep(1)
# Do some moves (encoder counts)
# or
# stage.move_absolute(12345)

In [33]:
# See the position (in encoder counts)
print(stage.status["position"])

345550


In [23]:
stage.status["moving_forward"]

False

In [27]:
stage.close()